In [1]:

### Need to add total innings in phases of the game.

import os, json
import numpy as np, pandas as pd

from pprint import pprint
from datetime import datetime as dt


In [2]:

csv_location = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\ipl_json_bbb\\'
exp_location = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\exp\\'
loc_lups = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\lineups\\'


In [3]:

print(csv_location)
print()
print(exp_location)
print()
print(dt.now().date())


C:\Users\karan\Documents\Cricket-Analytics\IPL\Data\ipl_json_bbb\

C:\Users\karan\Documents\Cricket-Analytics\IPL\Data\exp\

2023-05-08


In [4]:

folder_path = csv_location

# Get a list of all files and directories in the folder
all_files_and_dirs = os.listdir(folder_path)

# Filter the list to include only files (exclude directories)
files_only = [f for f in all_files_and_dirs if os.path.isfile(os.path.join(folder_path, f))]

# Print total # of files
print(len(files_only))

# Print names of first few files 
pprint(files_only[:5])


49
['2023_10_Sunrisers Hyderabad vs Lucknow Super Giants.csv',
 '2023_11_Rajasthan Royals vs Delhi Capitals.csv',
 '2023_12_Mumbai Indians vs Chennai Super Kings.csv',
 '2023_13_Gujarat Titans vs Kolkata Knight Riders.csv',
 '2023_14_Punjab Kings vs Sunrisers Hyderabad.csv']


In [5]:

files_list = files_only
total_files = len(files_list)

#print(np.arange(total_files))
print(total_files)


49


In [6]:

#All file numbers > 1359475 are #2023 IPL onwards
#All file numbers > 1304047 are #2022 IPL onwards


In [7]:

# initialize an empty DataFrame to store all the data
bbb_master_df = pd.DataFrame()

# iterate through each file in the directory
for file_name in os.listdir(csv_location):
    if file_name.endswith('.csv'):
        # read the CSV file into a DataFrame
        file_path = os.path.join(csv_location, file_name)
        df_temp = pd.read_csv(file_path)
        
        # append the data to the master DataFrame
        bbb_master_df = bbb_master_df.append(df_temp)

# print the master DataFrame
print(bbb_master_df.shape)


(11951, 20)


In [8]:

print(bbb_master_df.columns.values.tolist())


['inn', 'bat_tm', 'batnum', 'bat', 'ball', 'over_num', 'bow', 'runs', 'wkt', 'wkt_how', 'b.cnt', 'f.ov.powp', 'f.ov.midd', 'f.ov.deat', 'm.grnd', 'm.date', 'ss', 'm.type', 'm.num', 'm.vs']


In [9]:

display(bbb_master_df.head(3))


,inn,bat_tm,batnum,bat,ball,over_num,bow,runs,wkt,wkt_how,b.cnt,f.ov.powp,f.ov.midd,f.ov.deat,m.grnd,m.date,ss,m.type,m.num,m.vs
0,1,Sunrisers Hyderabad,1,Anmolpreet Singh,0,0,KR Mayers,1,0,NaN,1,1,0,0,Lucknow,2023-04-07,2023,T20,10,Sunrisers Hyderabad vs Lucknow Super Giants
1,1,Sunrisers Hyderabad,2,MA Agarwal,1,0,KR Mayers,1,0,NaN,1,1,0,0,Lucknow,2023-04-07,2023,T20,10,Sunrisers Hyderabad vs Lucknow Super Giants
2,1,Sunrisers Hyderabad,1,Anmolpreet Singh,2,0,KR Mayers,1,0,NaN,1,1,0,0,Lucknow,2023-04-07,2023,T20,10,Sunrisers Hyderabad vs Lucknow Super Giants


In [10]:

bbb_master_df.describe()


,inn,batnum,ball,over_num,runs,wkt,wkt_how,b.cnt,f.ov.powp,f.ov.midd,f.ov.deat,ss,m.num
count,11951.000000,11951.000000,11951.000000,11951.000000,11951.000000,11951.000000,0.0,11951.000000,11951.000000,11951.000000,11951.000000,11951.0,11951.000000
mean,1.487658,3.793992,2.640197,9.331688,1.351686,0.052548,NaN,0.960338,0.307589,0.506736,0.185675,2023.0,25.067609
std,0.499869,2.259796,1.829348,5.709928,1.716347,0.223138,NaN,0.195172,0.461515,0.499976,0.388860,0.0,14.203369
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,2023.0,1.000000
25%,1.000000,2.000000,1.000000,4.000000,0.000000,0.000000,NaN,1.000000,0.000000,0.000000,0.000000,2023.0,13.000000
50%,1.000000,3.000000,3.000000,9.000000,1.000000,0.000000,NaN,1.000000,0.000000,1.000000,0.000000,2023.0,25.000000
75%,2.000000,5.000000,4.000000,14.000000,1.000000,0.000000,NaN,1.000000,1.000000,1.000000,0.000000,2023.0,37.000000
max,2.000000,11.000000,10.000000,19.000000,6.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,2023.0,50.000000


#### Add bow_tm field for use

In [11]:

# Create a dictionary with the match number ('m.num') as the key and the team names as the values
team_mapping = bbb_master_df.groupby('m.num')['bat_tm'].unique().apply(list).to_dict()

# Define a function to switch the team name
def switch_team_name(team, match_num):
    teams = team_mapping[match_num]
    return teams[0] if teams[1] == team else teams[1]

# Add a new column 'bow_team' in the bowling_match_level_df
bbb_master_df = bbb_master_df.copy()
bbb_master_df['bow_tm'] = bbb_master_df.apply(lambda x: switch_team_name(x['bat_tm'], x['m.num']), axis=1)


In [12]:

print(len(bbb_master_df['m.num'].unique()))
#print(pre_master_df['m.vs'].unique())


49



#### Add the bowling and batting flags


In [13]:

print(bbb_master_df.columns.values.tolist())


['inn', 'bat_tm', 'batnum', 'bat', 'ball', 'over_num', 'bow', 'runs', 'wkt', 'wkt_how', 'b.cnt', 'f.ov.powp', 'f.ov.midd', 'f.ov.deat', 'm.grnd', 'm.date', 'ss', 'm.type', 'm.num', 'm.vs', 'bow_tm']


In [14]:

#### Add the linups data to the pstats dataset before more merges

lups = pd.read_excel(loc_lups + 'Tags.xlsx')

print(lups.shape)
print(lups.columns.values.tolist())


(187, 9)
['tm', 'name', 'fname', 'battype', 'bowltype', 'bowlarm', 'bowlcat', 'bowlcat.l2', 'lpos']


In [15]:

display(lups.tm.unique())


array(['Chennai Super Kings', 'Delhi Capitals', 'Gujarat Titans',
       'Kolkata Knight Riders', 'Lucknow Super Giants', 'Mumbai Indians',
       'Punjab Kings', 'Rajasthan Royals', 'Royal Challengers Bangalore',
       'Sunrisers Hyderabad'], dtype=object)

In [16]:

batcols = ['tm','name','fname','battype']

pre_master_df = pd.merge(lups[batcols], bbb_master_df, how = 'right', right_on = ['bat_tm','bat'], left_on = ['tm','name'])

print(pre_master_df.shape)
print()
print(pre_master_df.columns.values.tolist())


(11951, 25)

['tm', 'name', 'fname', 'battype', 'inn', 'bat_tm', 'batnum', 'bat', 'ball', 'over_num', 'bow', 'runs', 'wkt', 'wkt_how', 'b.cnt', 'f.ov.powp', 'f.ov.midd', 'f.ov.deat', 'm.grnd', 'm.date', 'ss', 'm.type', 'm.num', 'm.vs', 'bow_tm']


In [17]:

bowcols = ['tm','name','fname','bowltype','bowlarm','bowlcat','bowlcat.l2']

master_df = pd.merge(pre_master_df \
                                        , lups[bowcols]\
                                        , how = 'left'\
                                        , left_on = ['bow_tm','bow']\
                                        , right_on = ['tm','name']\
                     ,suffixes = ['_bat','_bow'])

print(master_df.shape)
print()
print(master_df.columns.values.tolist())


(11951, 32)

['tm_bat', 'name_bat', 'fname_bat', 'battype', 'inn', 'bat_tm', 'batnum', 'bat', 'ball', 'over_num', 'bow', 'runs', 'wkt', 'wkt_how', 'b.cnt', 'f.ov.powp', 'f.ov.midd', 'f.ov.deat', 'm.grnd', 'm.date', 'ss', 'm.type', 'm.num', 'm.vs', 'bow_tm', 'tm_bow', 'name_bow', 'fname_bow', 'bowltype', 'bowlarm', 'bowlcat', 'bowlcat.l2']


In [18]:

print(master_df.bat_tm.unique())


['Sunrisers Hyderabad' 'Lucknow Super Giants' 'Rajasthan Royals'
 'Delhi Capitals' 'Mumbai Indians' 'Chennai Super Kings' 'Gujarat Titans'
 'Kolkata Knight Riders' 'Punjab Kings' 'Royal Challengers Bangalore']


In [19]:

display( master_df.groupby(['m.num', 'inn','m.vs', 'bat_tm','bow_tm']).agg({'runs':'count'}).reset_index() )


,m.num,inn,m.vs,bat_tm,bow_tm,runs
0,1,1,Chennai Super Kings vs Gujarat Titans,Chennai Super Kings,Gujarat Titans,121
1,1,2,Chennai Super Kings vs Gujarat Titans,Gujarat Titans,Chennai Super Kings,122
2,2,1,Punjab Kings vs Kolkata Knight Riders,Punjab Kings,Kolkata Knight Riders,123
3,2,2,Punjab Kings vs Kolkata Knight Riders,Kolkata Knight Riders,Punjab Kings,99
4,3,1,Lucknow Super Giants vs Delhi Capitals,Lucknow Super Giants,Delhi Capitals,125
...,...,...,...,...,...,...
93,48,2,Rajasthan Royals vs Gujarat Titans,Gujarat Titans,Rajasthan Royals,84
94,49,1,Mumbai Indians vs Chennai Super Kings,Mumbai Indians,Chennai Super Kings,122
95,49,2,Mumbai Indians vs Chennai Super Kings,Chennai Super Kings,Mumbai Indians,109
96,50,1,Royal Challengers Bangalore vs Delhi Capitals,Royal Challengers Bangalore,Delhi Capitals,125


In [20]:

#master_df.groupby(['over_num']).agg({'runs':'sum'}).reset_index()


In [21]:

#master_df.groupby(['f.ov.powp','f.ov.midd','f.ov.deat','batnum'], dropna = False).agg({'runs':'sum'}).reset_index()


In [22]:

#master_df.groupby(['f.ov.powp','f.ov.midd','f.ov.deat','over_num'], dropna = False).agg({'runs':'sum'}).reset_index().sort_values(by=['over_num'])


## Batting stats 

In [23]:

print(master_df.columns.values.tolist())


['tm_bat', 'name_bat', 'fname_bat', 'battype', 'inn', 'bat_tm', 'batnum', 'bat', 'ball', 'over_num', 'bow', 'runs', 'wkt', 'wkt_how', 'b.cnt', 'f.ov.powp', 'f.ov.midd', 'f.ov.deat', 'm.grnd', 'm.date', 'ss', 'm.type', 'm.num', 'm.vs', 'bow_tm', 'tm_bow', 'name_bow', 'fname_bow', 'bowltype', 'bowlarm', 'bowlcat', 'bowlcat.l2']


In [24]:

#print(df.shape)


In [25]:

groupbycols = ['bat_tm', 'batnum', 'bat', 'battype', 'm.num']

#bat_grouped_df = master_df.groupby(['bat_tm', 'batnum', 'bat', 'm.num'])
bat_grouped_df = master_df.groupby(groupbycols) # batnum unique as the m.num per player

def custom_agg(group):
    data = {
        'tr': group['runs'].sum(),
        'tbf': group['b.cnt'].sum(),
        'tbinn': group['m.num'].nunique(),
        'tout': group['wkt'].sum(),
        
        'tr_pp': (group['f.ov.powp'] * group['runs']).sum(),
        'tbf_pp': group['f.ov.powp'].sum(),
        'tout_pp': (group['wkt'] * group['f.ov.powp']).sum(),
        
        'tr_mid': (group['f.ov.midd'] * group['runs']).sum(),
        'tbf_mid': group['f.ov.midd'].sum(),
        'tout_mid': (group['wkt'] * group['f.ov.midd']).sum(),
        
        'tr_dth': (group['f.ov.deat'] * group['runs']).sum(),
        'tbf_dth': group['f.ov.deat'].sum(),
        'tout_dth': (group['wkt'] * group['f.ov.deat']).sum()
    }
    return pd.Series(data)

batting_match_level_df = bat_grouped_df.apply(custom_agg)

# Reset the index so that the primary key columns become regular columns
batting_match_level_df.reset_index(inplace=True)


In [26]:

#batting_match_level_df.sum()


In [27]:

print(batting_match_level_df.shape)


(798, 18)


In [28]:

display(batting_match_level_df.head(3))


,bat_tm,batnum,bat,battype,m.num,tr,tbf,tbinn,tout,tr_pp,tbf_pp,tout_pp,tr_mid,tbf_mid,tout_mid,tr_dth,tbf_dth,tout_dth
0,Chennai Super Kings,1,DP Conway,LHB,1,1,6,1,1,1,6,1,0,0,0,0,0,0
1,Chennai Super Kings,1,DP Conway,LHB,12,0,4,1,1,0,4,1,0,0,0,0,0,0
2,Chennai Super Kings,1,RD Gaikwad,RHB,6,57,31,1,1,46,21,0,11,11,1,0,0,0


In [29]:

# Add points columns to the original DataFrame

batting_match_level_df['f.cp.pts30'] = batting_match_level_df['tr'].apply(lambda x: 4 if x >= 30 else 0)
batting_match_level_df['f.cp.pts50'] = batting_match_level_df['tr'].apply(lambda x: 8 if x >= 50 else 0)
batting_match_level_df['f.cp.pts100'] = batting_match_level_df['tr'].apply(lambda x: 16 if x >= 100 else 0)
batting_match_level_df['f.cp.pts0'] = batting_match_level_df['tr'].apply(lambda x: -2 if x == 0 else 0)


In [30]:

batting_match_level_df.describe()


,batnum,m.num,tr,tbf,tbinn,tout,tr_pp,tbf_pp,tout_pp,tr_mid,tbf_mid,tout_mid,tr_dth,tbf_dth,tout_dth,f.cp.pts30,f.cp.pts50,f.cp.pts100,f.cp.pts0
count,798.000000,798.000000,798.000000,798.000000,798.0,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000
mean,4.830827,24.875940,20.215539,14.354637,1.0,0.783208,5.981203,4.588972,0.179198,9.857143,7.577694,0.358396,4.377193,2.780702,0.245614,1.027569,0.992481,0.060150,-0.205514
std,2.710674,14.168582,21.537715,12.971673,0.0,0.430189,10.777592,6.956324,0.383759,14.174469,9.282320,0.482438,8.066495,4.331412,0.433624,1.748774,2.638855,0.979791,0.607663
min,1.000000,1.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000
25%,3.000000,13.000000,3.000000,4.000000,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,25.000000,13.000000,10.000000,1.0,1.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.000000,37.000000,30.000000,21.000000,1.0,1.000000,8.000000,8.000000,0.000000,16.750000,13.000000,1.000000,6.000000,5.000000,0.000000,4.000000,0.000000,0.000000,0.000000
max,11.000000,50.000000,124.000000,66.000000,1.0,2.000000,54.000000,28.000000,1.000000,66.000000,40.000000,2.000000,49.000000,23.000000,2.000000,4.000000,8.000000,16.000000,0.000000


In [31]:

print(batting_match_level_df.shape)
print()
print(batting_match_level_df.columns.values.tolist())


(798, 22)

['bat_tm', 'batnum', 'bat', 'battype', 'm.num', 'tr', 'tbf', 'tbinn', 'tout', 'tr_pp', 'tbf_pp', 'tout_pp', 'tr_mid', 'tbf_mid', 'tout_mid', 'tr_dth', 'tbf_dth', 'tout_dth', 'f.cp.pts30', 'f.cp.pts50', 'f.cp.pts100', 'f.cp.pts0']


### Batting Transpose

In [32]:

df_bat_transpose = batting_match_level_df.copy()
df_bat_transpose['game.num'] = 'G' + df_bat_transpose['m.num'].astype(str).str.zfill(2)

df_bat_transpose.sort_values(by = 'm.num', ascending = False)

batting_gwise = df_bat_transpose.pivot(index= ['bat_tm','bat','battype'], columns= ['game.num'], values= ['tr']).reset_index()

# Fixing multi-level columns
#batting_gwise.columns = batting_gwise.columns.map(lambda x: f'{x[1]}.{x[0]}')

# Flatten only multi-level column index
batting_gwise.columns = batting_gwise.columns.map( lambda x: f'{x[0]}.{x[1]}' if len(x) > 1 else x[0] )

# Reset index
batting_gwise = batting_gwise.reset_index(drop = True).rename(columns = {'bat_tm.':'bat_tm','bat.':'bat', 'battype.':'battype'})


In [33]:

print(batting_gwise.shape)
print()
print(batting_gwise.columns.values.tolist())


(152, 52)

['bat_tm', 'bat', 'battype', 'tr.G01', 'tr.G02', 'tr.G03', 'tr.G04', 'tr.G05', 'tr.G06', 'tr.G07', 'tr.G08', 'tr.G09', 'tr.G10', 'tr.G11', 'tr.G12', 'tr.G13', 'tr.G14', 'tr.G15', 'tr.G16', 'tr.G17', 'tr.G18', 'tr.G19', 'tr.G20', 'tr.G21', 'tr.G22', 'tr.G23', 'tr.G24', 'tr.G25', 'tr.G26', 'tr.G27', 'tr.G28', 'tr.G29', 'tr.G30', 'tr.G31', 'tr.G32', 'tr.G33', 'tr.G34', 'tr.G35', 'tr.G36', 'tr.G37', 'tr.G38', 'tr.G39', 'tr.G40', 'tr.G41', 'tr.G42', 'tr.G43', 'tr.G44', 'tr.G46', 'tr.G47', 'tr.G48', 'tr.G49', 'tr.G50']


In [34]:

display(batting_gwise)


,bat_tm,bat,battype,tr.G01,tr.G02,tr.G03,tr.G04,tr.G05,tr.G06,tr.G07,...,tr.G40,tr.G41,tr.G42,tr.G43,tr.G44,tr.G46,tr.G47,tr.G48,tr.G49,tr.G50
0,Chennai Super Kings,AM Rahane,RHB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,NaN
1,Chennai Super Kings,AT Rayudu,RHB,12.0,NaN,NaN,NaN,NaN,27.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN
2,Chennai Super Kings,BA Stokes,LHB,7.0,NaN,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chennai Super Kings,DP Conway,LHB,1.0,NaN,NaN,NaN,NaN,47.0,NaN,...,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN,44.0,NaN
4,Chennai Super Kings,MJ Santner,LHB,1.0,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Sunrisers Hyderabad,M Markande,RHB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
148,Sunrisers Hyderabad,MA Agarwal,RHB,NaN,NaN,NaN,27.0,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN
149,Sunrisers Hyderabad,RA Tripathi,RHB,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN
150,Sunrisers Hyderabad,Umran Malik,RHB,NaN,NaN,NaN,19.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Bowling stats

In [35]:

print(master_df.columns.values.tolist())


['tm_bat', 'name_bat', 'fname_bat', 'battype', 'inn', 'bat_tm', 'batnum', 'bat', 'ball', 'over_num', 'bow', 'runs', 'wkt', 'wkt_how', 'b.cnt', 'f.ov.powp', 'f.ov.midd', 'f.ov.deat', 'm.grnd', 'm.date', 'ss', 'm.type', 'm.num', 'm.vs', 'bow_tm', 'tm_bow', 'name_bow', 'fname_bow', 'bowltype', 'bowlarm', 'bowlcat', 'bowlcat.l2']


In [36]:

### DICTIONARY CREATION

### ROLLING UP THE DATASET
def custom_bowling_agg(group):
    data = {
        'tbb': group['b.cnt'].sum(),
        'tbrc': group['runs'].sum(),
        'tbwkt': group['wkt'].sum(),
        'tbov': group['over_num'].nunique(),
        
        'tbrc_powp': (group['f.ov.powp'] * group['runs']).sum(),
        'tbrc_mid': (group['f.ov.midd'] * group['runs']).sum(),
        'tbrc_dth': (group['f.ov.deat'] * group['runs']).sum(),
        
        'tbwkt_powp': (group['f.ov.powp'] * group['wkt']).sum(),
        'tbwkt_mid': (group['f.ov.midd'] * group['wkt']).sum(),
        'tbwkt_dth': (group['f.ov.deat'] * group['wkt']).sum()
    }
    return pd.Series(data)

bow_grouped_df = master_df.groupby(['bow_tm', 'bow', 'bowltype', 'bowlarm', 'bowlcat', 'bowlcat.l2', 'm.num'])

bowling_match_level_df_pre = bow_grouped_df.apply(custom_bowling_agg)
bowling_match_level_df_pre.reset_index(inplace=True)

print(master_df.shape)
print(bowling_match_level_df_pre.shape)


(11951, 32)
(597, 17)


In [37]:

bowling_match_level_df_pre.describe()


,m.num,tbb,tbrc,tbwkt,tbov,tbrc_powp,tbrc_mid,tbrc_dth,tbwkt_powp,tbwkt_mid,tbwkt_dth
count,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000
mean,25.145729,19.093802,26.825796,1.048576,3.204355,7.966499,13.050251,5.809045,0.241206,0.480737,0.326633
std,14.222720,6.205479,10.983352,1.056771,1.025733,8.168047,10.668785,7.882764,0.526586,0.738478,0.679650
min,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.000000,16.000000,19.000000,0.000000,3.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000
50%,25.000000,24.000000,27.000000,1.000000,4.000000,7.000000,12.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,24.000000,34.000000,2.000000,4.000000,14.000000,20.000000,11.000000,0.000000,1.000000,0.000000
max,50.000000,24.000000,69.000000,5.000000,4.000000,37.000000,49.000000,38.000000,4.000000,4.000000,4.000000


In [38]:

print(master_df.shape)

# master_df for each phase of the game
powp_df = master_df[master_df['f.ov.powp'] != 0]
mid_df = master_df[master_df['f.ov.midd'] != 0]
dth_df = master_df[master_df['f.ov.deat'] != 0]

print(powp_df.shape)
print(mid_df.shape)
print(dth_df.shape)

# Group the DataFrames by the bowler, batting team, and match number
#powp_grouped = powp_df.groupby(['bow_tm', 'bow', 'm.num']).agg(tot_ov_powp = ('over_num','nunique')).reset_index()
powp_grouped = powp_df.groupby(['bow_tm', 'bow', 'm.num']).agg({'over_num':'nunique'}).reset_index().rename(columns = {'over_num':'tbov.powp'})
mid_grouped = mid_df.groupby(['bow_tm', 'bow', 'm.num']).agg({'over_num':'nunique'}).reset_index().rename(columns = {'over_num':'tbov.mid'})
dth_grouped = dth_df.groupby(['bow_tm', 'bow', 'm.num']).agg({'over_num':'nunique'}).reset_index().rename(columns = {'over_num':'tbov.dth'})


(11951, 32)
(3676, 32)
(6056, 32)
(2219, 32)


In [39]:

# Reset the index of bowling_match_level_df
#bow_grouped_df.reset_index()

bowling_match_level_df = bowling_match_level_df_pre.reset_index()

# Merge the unique overs bowled in each phase back to the bowling_match_level_df
bowling_match_level_df = bowling_match_level_df.merge(powp_grouped, on=['bow_tm', 'bow', 'm.num'], how='left')
bowling_match_level_df = bowling_match_level_df.merge(mid_grouped, on=['bow_tm', 'bow', 'm.num'], how='left')
bowling_match_level_df = bowling_match_level_df.merge(dth_grouped, on=['bow_tm', 'bow', 'm.num'], how='left')


In [40]:

print(bowling_match_level_df.columns.values.tolist())


['index', 'bow_tm', 'bow', 'bowltype', 'bowlarm', 'bowlcat', 'bowlcat.l2', 'm.num', 'tbb', 'tbrc', 'tbwkt', 'tbov', 'tbrc_powp', 'tbrc_mid', 'tbrc_dth', 'tbwkt_powp', 'tbwkt_mid', 'tbwkt_dth', 'tbov.powp', 'tbov.mid', 'tbov.dth']


In [41]:

# BOWLING

bowling_match_level_df['f.cp.pts3w'] = bowling_match_level_df['tbwkt'].apply(lambda x : 4 if x >= 3 else 0 )
bowling_match_level_df['f.cp.pts4w'] = bowling_match_level_df['tbwkt'].apply(lambda x : 8 if x >= 4 else 0 )
bowling_match_level_df['f.cp.pts5w'] = bowling_match_level_df['tbwkt'].apply(lambda x : 16 if x >= 5 else 0 )

bowling_match_level_df['tbecon'] = ( bowling_match_level_df['tbrc'] * 6 ) / bowling_match_level_df['tbb']

bowling_match_level_df['f.cp.ptsecon'] = bowling_match_level_df.apply(lambda x : (2*6)/x.tbb if (5 <= x.tbecon) & ( x.tbecon < 6) \
                                                            else ( (4*6)/x.tbb if (5 <= x.tbecon) & ( x.tbecon < 6) else 0 )\
                                                , axis =1)


In [42]:

display(bowling_match_level_df.head(3))


,index,bow_tm,bow,bowltype,bowlarm,bowlcat,bowlcat.l2,m.num,tbb,tbrc,...,tbwkt_mid,tbwkt_dth,tbov.powp,tbov.mid,tbov.dth,f.cp.pts3w,f.cp.pts4w,f.cp.pts5w,tbecon,f.cp.ptsecon
0,0,Chennai Super Kings,Akash Singh,LAFM,Left,Pace,medium,17,24,39,...,1,1,2.0,1.0,1.0,0,0,0,9.750000,0.000000
1,1,Chennai Super Kings,Akash Singh,LAFM,Left,Pace,medium,24,18,35,...,0,0,3.0,NaN,NaN,0,0,0,11.666667,0.000000
2,2,Chennai Super Kings,Akash Singh,LAFM,Left,Pace,medium,29,18,17,...,0,0,3.0,NaN,NaN,0,0,0,5.666667,0.666667


In [43]:

bowling_match_level_df.describe()


,index,m.num,tbb,tbrc,tbwkt,tbov,tbrc_powp,tbrc_mid,tbrc_dth,tbwkt_powp,tbwkt_mid,tbwkt_dth,tbov.powp,tbov.mid,tbov.dth,f.cp.pts3w,f.cp.pts4w,f.cp.pts5w,tbecon,f.cp.ptsecon
count,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,367.000000,477.000000,281.000000,597.000000,597.000000,597.000000,597.000000,597.000000
mean,298.000000,25.145729,19.093802,26.825796,1.048576,3.204355,7.966499,13.050251,5.809045,0.241206,0.480737,0.326633,1.596730,2.035639,1.266904,0.395310,0.214405,0.026801,8.794621,0.052344
std,172.483333,14.222720,6.205479,10.983352,1.056771,1.025733,8.168047,10.668785,7.882764,0.526586,0.738478,0.679650,0.670038,1.017075,0.443131,1.194722,1.293087,0.654836,3.276640,0.199931
min,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,149.000000,13.000000,16.000000,19.000000,0.000000,3.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,6.500000,0.000000
50%,298.000000,25.000000,24.000000,27.000000,1.000000,4.000000,7.000000,12.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,8.250000,0.000000
75%,447.000000,37.000000,24.000000,34.000000,2.000000,4.000000,14.000000,20.000000,11.000000,0.000000,1.000000,0.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,10.500000,0.000000
max,596.000000,50.000000,24.000000,69.000000,5.000000,4.000000,37.000000,49.000000,38.000000,4.000000,4.000000,4.000000,3.000000,4.000000,2.000000,4.000000,8.000000,16.000000,25.000000,2.000000


In [44]:

# QC
# bowling_match_level_df[bowling_match_level_df['tov.powp'] == 4]


## Bowling Transpose

In [45]:

df_bow_transpose = bowling_match_level_df.copy()
df_bow_transpose['game.num'] = 'G' + df_bow_transpose['m.num'].astype(str).str.zfill(2)

df_bow_transpose.sort_values(by = 'game.num', ascending = True)

bowling_gwise = df_bow_transpose.pivot( index= ['bow_tm','bow'], columns= ['game.num'], values= ['tbwkt'] ).reset_index()

# Fixing multi-level columns
#bowling_gwise.columns = bowling_gwise.columns.map(lambda x: f'{x[1]}.{x[0]}')

# Flatten only multi-level column index
bowling_gwise.columns = bowling_gwise.columns.map( lambda x: f'{x[0]}.{x[1]}' if len(x) > 1 else x[0] )

# Reset index
bowling_gwise = bowling_gwise.reset_index(drop = True).rename(columns = {'bow_tm.':'bow_tm','bow.':'bow'})


In [46]:

print(bowling_gwise.shape)
print()
print(bowling_gwise.columns.values.tolist())


(121, 51)

['bow_tm', 'bow', 'tbwkt.G01', 'tbwkt.G02', 'tbwkt.G03', 'tbwkt.G04', 'tbwkt.G05', 'tbwkt.G06', 'tbwkt.G07', 'tbwkt.G08', 'tbwkt.G09', 'tbwkt.G10', 'tbwkt.G11', 'tbwkt.G12', 'tbwkt.G13', 'tbwkt.G14', 'tbwkt.G15', 'tbwkt.G16', 'tbwkt.G17', 'tbwkt.G18', 'tbwkt.G19', 'tbwkt.G20', 'tbwkt.G21', 'tbwkt.G22', 'tbwkt.G23', 'tbwkt.G24', 'tbwkt.G25', 'tbwkt.G26', 'tbwkt.G27', 'tbwkt.G28', 'tbwkt.G29', 'tbwkt.G30', 'tbwkt.G31', 'tbwkt.G32', 'tbwkt.G33', 'tbwkt.G34', 'tbwkt.G35', 'tbwkt.G36', 'tbwkt.G37', 'tbwkt.G38', 'tbwkt.G39', 'tbwkt.G40', 'tbwkt.G41', 'tbwkt.G42', 'tbwkt.G43', 'tbwkt.G44', 'tbwkt.G46', 'tbwkt.G47', 'tbwkt.G48', 'tbwkt.G49', 'tbwkt.G50']


In [47]:

display(bowling_gwise)


,bow_tm,bow,tbwkt.G01,tbwkt.G02,tbwkt.G03,tbwkt.G04,tbwkt.G05,tbwkt.G06,tbwkt.G07,tbwkt.G08,...,tbwkt.G40,tbwkt.G41,tbwkt.G42,tbwkt.G43,tbwkt.G44,tbwkt.G46,tbwkt.G47,tbwkt.G48,tbwkt.G49,tbwkt.G50
0,Chennai Super Kings,Akash Singh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chennai Super Kings,BA Stokes,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chennai Super Kings,D Pretorius,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chennai Super Kings,DL Chahar,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
4,Chennai Super Kings,M Pathirana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Sunrisers Hyderabad,M Markande,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
117,Sunrisers Hyderabad,Mayank Dagar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,Sunrisers Hyderabad,T Natarajan,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
119,Sunrisers Hyderabad,Umran Malik,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## MERGE

In [48]:

### MERGE BAT AND BOW DF

# Reset the index for both DataFrames to prepare for merging
batmldf = batting_match_level_df.reset_index(drop = True)
bowmldf = bowling_match_level_df.reset_index(drop = True)

#dropcols = ['bow_tm','bow']
dropcols = []

# Merge the batting figures with the bowling stats
merged_predf = pd.merge( batmldf, bowmldf, 
                     left_on=['bat_tm', 'bat', 'm.num'],
                     right_on=['bow_tm', 'bow', 'm.num'],
                     how='outer', 
                     suffixes=('_bat', '_bowl'))#.rename(columns = {'bat_tm':'tm', 'bat':'name' }).drop(columns = dropcols)

print(merged_predf.shape)
print(merged_predf.columns.values.tolist())

# Replace NaN values with 0 in the merged DataFrame
#merged_predf.fillna(0, inplace=True)



(1131, 47)
['bat_tm', 'batnum', 'bat', 'battype', 'm.num', 'tr', 'tbf', 'tbinn', 'tout', 'tr_pp', 'tbf_pp', 'tout_pp', 'tr_mid', 'tbf_mid', 'tout_mid', 'tr_dth', 'tbf_dth', 'tout_dth', 'f.cp.pts30', 'f.cp.pts50', 'f.cp.pts100', 'f.cp.pts0', 'index', 'bow_tm', 'bow', 'bowltype', 'bowlarm', 'bowlcat', 'bowlcat.l2', 'tbb', 'tbrc', 'tbwkt', 'tbov', 'tbrc_powp', 'tbrc_mid', 'tbrc_dth', 'tbwkt_powp', 'tbwkt_mid', 'tbwkt_dth', 'tbov.powp', 'tbov.mid', 'tbov.dth', 'f.cp.pts3w', 'f.cp.pts4w', 'f.cp.pts5w', 'tbecon', 'f.cp.ptsecon']


In [49]:

display(merged_predf.head(5))


,bat_tm,batnum,bat,battype,m.num,tr,tbf,tbinn,tout,tr_pp,...,tbwkt_mid,tbwkt_dth,tbov.powp,tbov.mid,tbov.dth,f.cp.pts3w,f.cp.pts4w,f.cp.pts5w,tbecon,f.cp.ptsecon
0,Chennai Super Kings,1.0,DP Conway,LHB,1,1.0,6.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chennai Super Kings,1.0,DP Conway,LHB,12,0.0,4.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chennai Super Kings,1.0,RD Gaikwad,RHB,6,57.0,31.0,1.0,1.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chennai Super Kings,1.0,RD Gaikwad,RHB,17,8.0,10.0,1.0,1.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chennai Super Kings,1.0,RD Gaikwad,RHB,24,3.0,6.0,1.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:

# Function to combine two columns with condition
def combine_columns(row, col1, col2):
    return row[col1] if pd.notna(row[col1]) and row[col1] != '' else row[col2]

# Create a new column using the custom function
merged_predf['tm'] = merged_predf.apply(lambda row: combine_columns(row, 'bat_tm', 'bow_tm'), axis=1)
merged_predf['name'] = merged_predf.apply(lambda row: combine_columns(row, 'bat', 'bow'), axis=1)


In [51]:

display(merged_predf.tail(5))


,bat_tm,batnum,bat,battype,m.num,tr,tbf,tbinn,tout,tr_pp,...,tbov.powp,tbov.mid,tbov.dth,f.cp.pts3w,f.cp.pts4w,f.cp.pts5w,tbecon,f.cp.ptsecon,tm,name
1126,NaN,NaN,NaN,NaN,29,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,1.0,0.0,0.0,0.0,6.0,0.0,Sunrisers Hyderabad,Umran Malik
1127,NaN,NaN,NaN,NaN,34,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,NaN,0.0,0.0,0.0,7.0,0.0,Sunrisers Hyderabad,Umran Malik
1128,NaN,NaN,NaN,NaN,40,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,0.0,0.0,0.0,22.0,0.0,Sunrisers Hyderabad,Umran Malik
1129,NaN,NaN,NaN,NaN,14,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,0.0,0.0,0.0,6.0,0.0,Sunrisers Hyderabad,Washington Sundar
1130,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,NaN,0.0,0.0,0.0,10.0,0.0,Sunrisers Hyderabad,Washington Sundar


In [52]:

#merged_df.to_csv(exp_location + 'merged.csv', index = False)


### MAIN RUP

In [53]:


# Merged PreDF has the both the batting and bowling stats for each player and can be easily rolled up to see the results


In [54]:

#### Rollup at a Player Level

# Replace NaN values with False in the mask
#mask = merged_predf.tm.str.contains(team_for_code).fillna(False)

# Apply the mask to filter the DataFrame
#merged_for_team_only = merged_df[merged_df.tm.str.contains(team_for_code)]
#merged_for_team_only = merged_predf[mask]
merged_for_team_only = merged_predf

tstats = merged_for_team_only.groupby(['tm']).agg({
    #'batnum': np.min,
    'm.num': 'nunique',
    'tr': 'sum',
    'tbf': 'sum',
    #'tbinn': 'sum',
    'tout': 'sum',
    'tr_pp': 'sum',
    'tbf_pp': 'sum',
    'tout_pp': 'sum',
    'tr_mid': 'sum',
    'tbf_mid': 'sum',
    'tout_mid': 'sum',
    'tr_dth': 'sum',
    'tbf_dth': 'sum',
    'tout_dth': 'sum',
    #'f.cp.pts30': 'sum',
    #'f.cp.pts50': 'sum',
    #'f.cp.pts100': 'sum',
    #'f.cp.pts0': 'sum',    
    'tbb': 'sum',
    'tbrc': 'sum',
    'tbwkt': 'sum',
    'tbov': 'sum',
    'tbov.powp':'sum', 
    'tbov.mid':'sum', 
    'tbov.dth':'sum',
    'tbrc_powp': 'sum',
    'tbrc_mid': 'sum',
    'tbrc_dth': 'sum',
    'tbwkt_powp': 'sum',
    'tbwkt_mid': 'sum',
    'tbwkt_dth': 'sum'
}).reset_index().rename(columns = {'batnum' : 'batnum_min','m.num':'tinn'})

print(tstats.shape)


(10, 27)


In [55]:

print(tstats.columns.values.tolist())


['tm', 'tinn', 'tr', 'tbf', 'tout', 'tr_pp', 'tbf_pp', 'tout_pp', 'tr_mid', 'tbf_mid', 'tout_mid', 'tr_dth', 'tbf_dth', 'tout_dth', 'tbb', 'tbrc', 'tbwkt', 'tbov', 'tbov.powp', 'tbov.mid', 'tbov.dth', 'tbrc_powp', 'tbrc_mid', 'tbrc_dth', 'tbwkt_powp', 'tbwkt_mid', 'tbwkt_dth']


In [56]:

display(tstats.transpose())


,0,1,2,3,4,5,6,7,8,9
tm,Chennai Super Kings,Delhi Capitals,Gujarat Titans,Kolkata Knight Riders,Lucknow Super Giants,Mumbai Indians,Punjab Kings,Rajasthan Royals,Royal Challengers Bangalore,Sunrisers Hyderabad
tinn,10,10,10,10,9,10,10,10,10,9
tr,1764.0,1447.0,1571.0,1713.0,1443.0,1737.0,1727.0,1702.0,1662.0,1366.0
tbf,1167.0,1152.0,1134.0,1176.0,1055.0,1176.0,1186.0,1183.0,1164.0,1062.0
tout,50.0,74.0,46.0,73.0,64.0,60.0,72.0,65.0,62.0,59.0
tr_pp,539.0,445.0,463.0,462.0,409.0,488.0,505.0,537.0,538.0,387.0
tbf_pp,372.0,362.0,377.0,380.0,343.0,373.0,380.0,374.0,364.0,337.0
tout_pp,7.0,20.0,14.0,22.0,13.0,14.0,18.0,10.0,10.0,15.0
tr_mid,847.0,710.0,736.0,890.0,747.0,872.0,834.0,745.0,818.0,667.0
tbf_mid,612.0,610.0,600.0,629.0,557.0,613.0,624.0,625.0,618.0,559.0


In [57]:
# Now check the gwise battin and bowling stats to append to pstats

## Percentage split

In [58]:

pct_splits = tstats.copy()


In [59]:

pct_splits['tr%.pp'] = round(100*pct_splits['tr_pp'] / ( pct_splits['tr'] ),0)
pct_splits['tr%.mid'] = round(100*pct_splits['tr_mid'] / ( pct_splits['tr'] ),0)
pct_splits['tr%.dth'] = round(100*pct_splits['tr_dth'] / ( pct_splits['tr'] ),0)


In [60]:

pct_splits['tbov%.pp'] = round(100*pct_splits['tbov.powp'] / ( pct_splits['tbov'] ),0)
pct_splits['tbov%.mid'] = round(100*pct_splits['tbov.mid'] / ( pct_splits['tbov'] ),0)
pct_splits['tbov%.dth'] = round(100*pct_splits['tbov.dth'] / ( pct_splits['tbov'] ),0)


In [61]:

pct_splits['tbwkt%.pp'] = round(100*pct_splits['tbwkt_powp'] / ( pct_splits['tbwkt'] ),0)
pct_splits['tbwkt%.mid'] = round(100*pct_splits['tbwkt_mid'] / ( pct_splits['tbwkt'] ),0)
pct_splits['tbwkt%.dth'] = round(100*pct_splits['tbwkt_dth'] / ( pct_splits['tbwkt'] ),0)


In [62]:

print(pct_splits.shape)
print()
print(pct_splits.columns.values.tolist())
print()


(10, 36)

['tm', 'tinn', 'tr', 'tbf', 'tout', 'tr_pp', 'tbf_pp', 'tout_pp', 'tr_mid', 'tbf_mid', 'tout_mid', 'tr_dth', 'tbf_dth', 'tout_dth', 'tbb', 'tbrc', 'tbwkt', 'tbov', 'tbov.powp', 'tbov.mid', 'tbov.dth', 'tbrc_powp', 'tbrc_mid', 'tbrc_dth', 'tbwkt_powp', 'tbwkt_mid', 'tbwkt_dth', 'tr%.pp', 'tr%.mid', 'tr%.dth', 'tbov%.pp', 'tbov%.mid', 'tbov%.dth', 'tbwkt%.pp', 'tbwkt%.mid', 'tbwkt%.dth']



In [63]:


pdetails = ['name','bowltype','bowlarm','bowlcat','bowlcat.l2']

tr_phs = ['tr_pp','tr_mid','tr_dth']
tbf_phs = ['tbf_pp','tbf_mid','tbf_dth']
tout_phs = ['tout_pp','tout_mid','tout_dth']

tbov_phs = ['tbov.powp','tbov.mid','tbov.dth']
tbwkt_phs = ['tbwkt_powp','tbwkt_mid','tbwkt_dth']
tbrc_phs = ['tbrc_powp','tbrc_mid','tbrc_dth']

tbov_pct = ['tbov%.pp', 'tbov%.mid', 'tbov%.dth']

#f_cp = ['f.cp.pts30', 'f.cp.pts50', 'f.cp.pts100', 'f.cp.pts0']

#misc = ['bat_tm','bow_tm']


In [64]:

dfexp = pct_splits.drop( columns = ['tbf'] + tbf_phs + ['tbov'] + tbov_phs + tbov_pct )
#dfexp = pct_splits.copy()


In [65]:

# Change the column order
new_order = ['tm', 'tinn', 'tr'\
             , 'tr_pp', 'tr_mid', 'tr_dth'\
             , 'tout_pp', 'tout_mid', 'tout_dth'\
             , 'tbb', 'tbrc', 'tbrc_powp', 'tbrc_mid', 'tbrc_dth'\
             , 'tbwkt', 'tbwkt_powp', 'tbwkt_mid', 'tbwkt_dth'\
             , 'tr%.pp', 'tr%.mid', 'tr%.dth'\
             , 'tbwkt%.pp', 'tbwkt%.mid', 'tbwkt%.dth']
expfile = dfexp[new_order].copy().transpose()



In [66]:

newindex = { 'tm' : 'team' , 'tinn': 'innings'\
            , 'tr': 'runs', 'tr_pp': 'runs in powerplay', 'tr_mid': 'runs in middle overs', 'tr_dth': 'total runs in death overs'
            , 'tout_pp': 'outs in powerplay', 'tout_mid': 'outs in middle overs', 'tout_dth': 'outs in death overs'
            , 'tbb': 'balls bowled'\
            , 'tbrc': 'run conceded', 'tbrc_powp': 'run conceded in powerplay', 'tbrc_mid': 'run conceded in middle overs', 'tbrc_dth': 'run conceded in death overs'
            , 'tbwkt': 'wickets taken', 'tbwkt_powp': 'wickets in powerplay', 'tbwkt_mid': 'wickets taken in middle overs' , 'tbwkt_dth': 'wickets taken in death overs'
            , 'tr%.pp': 'run % in powerplay', 'tr%.mid': 'run % in middle overs', 'tr%.dth': 'run % in death overs'
            , 'tbwkt%.pp': 'wicket % in powerplay', 'tbwkt%.mid': 'wicket % in middle overs', 'tbwkt%.dth': 'wicket % in death overs' }


In [67]:

expfile_renamed = expfile.rename(index=newindex)


In [68]:

exp_file = f'tstats_all.xlsx'
expfile_renamed.to_excel(exp_location + exp_file, index = True)


In [69]:

print(dt.now())


2023-05-08 12:14:04.468784


### Fantasy Points Totals

In [70]:
'''
#Create Fantasy pts crickpe version

pstats['bat_pts'] = master_df['rs'].fillna(0) + master_df['four'].fillna(0) + master_df['six'].fillna(0)*2 \
    + master_df['pts_for_thrty'].fillna(0) + master_df['pts_for_fifty'].fillna(0) + master_df['pts_for_hundy'].fillna(0) + master_df['pts_for_ducky'].fillna(0)

master_df['bow_pts'] = master_df['wkt'].fillna(0)*25 \
    + master_df['pts_for_3w'].fillna(0) + master_df['pts_for_4w'].fillna(0) + master_df['pts_for_5w'].fillna(0) \
        + master_df['pts_for_bow_econ'].fillna(0)

master_df['pts_crickpe'] = master_df['bat_pts'].fillna(0) + master_df['bow_pts'].fillna(0)
'''

"\n#Create Fantasy pts crickpe version\n\npstats['bat_pts'] = master_df['rs'].fillna(0) + master_df['four'].fillna(0) + master_df['six'].fillna(0)*2     + master_df['pts_for_thrty'].fillna(0) + master_df['pts_for_fifty'].fillna(0) + master_df['pts_for_hundy'].fillna(0) + master_df['pts_for_ducky'].fillna(0)\n\nmaster_df['bow_pts'] = master_df['wkt'].fillna(0)*25     + master_df['pts_for_3w'].fillna(0) + master_df['pts_for_4w'].fillna(0) + master_df['pts_for_5w'].fillna(0)         + master_df['pts_for_bow_econ'].fillna(0)\n\nmaster_df['pts_crickpe'] = master_df['bat_pts'].fillna(0) + master_df['bow_pts'].fillna(0)\n"

In [71]:

# Calculate strike rates for each phase of the game
#df_plyr_mtch['sr_powp'] = (df_plyr_mtch['tot_runs_powp'] / df_plyr_mtch['tot_balls_powp']) * 100
#df_plyr_mtch['sr_mid'] = (df_plyr_mtch['tot_runs_mid'] / df_plyr_mtch['tot_balls_mid']) * 100
#df_plyr_mtch['sr_dth'] = (df_plyr_mtch['tot_runs_dth'] / df_plyr_mtch['tot_balls_dth']) * 100
